# How do I _copy_ a file from the _Public Reference_ files?
### Overview
Files can be copied from other projects you are a member of or public reference. Here we focus on copying a file from _public reference_.

### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 
## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The _Api_ object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify platform {cgc, sbpla, etc}
prof = 'sbpla'


config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Copy a file from the Public Reference
We will first find our _source\_project_ (the Public Reference Files), then list the files within the source project<sup>1</sup>, and copy a file from **_source\_project_ -> _my\_project_**.

The critical information for this POST is the **file_id**. Note, you are allow to copy the same file as many times as you like. However, duplicates will be automatically have a prefix attached of (\_1\_, \_2\_, etc) depending on how many times you copy the file.

To make these results very obvious, use an empty project as your MY\_PROJECT (e.g. your <a href=projects_makeNew.ipynb> cookbook example project</a>) or change the _name_ in the _data_ dictionary to something like 'Dept of Awesome.copy'

<sup>1</sup> Files are only accessible **within** a project - here the Public Reference project (**warning** we may change this project name in the future).

In [ ]:
# [USER INPUT] Set your project name; source project ID; and files to copy here:
my_project_name = 'Keep on Smiling'
source_project_id = 'admin/sbg-public-data'        
files_list = ['WES_human_Illumina.pe_1.fastq',
             'WES_human_Illumina.pe_2.fastq'
             ]


# Double-check that your project exists
my_project = [p for p in api.projects.query(limit=100).all()
                  if p.name == my_project_name]
if not my_project:
    print('Your project (%s) not found, check spelling' % my_project_name)
    raise KeyboardInterrupt
else:
    my_project = my_project[0]
    
# LIST all file names in your project and file objects from source project
my_file_names = [f.name for f in 
    api.files.query(limit = 100, project = my_project).all()
]

source_files = api.files.query(
    limit = 100, project = source_project_id, names = files_list
)

if not source_files:
    print('Source files not found, check spelling and project id')
    raise KeyboardInterrupt
    
# Copy files if they don't already exist in my_project
for f in source_files:
    if f.name in my_file_names:
        print('File already exists in second project, skipping')
    else:
        print('File {} does not exist in {}; copying now'.format(
              f.name, my_project.name))

        new_file = f.copy(project = my_project, name = f.name)

        # re-list files in target project to verify the copy worked
        my_files = [f.name for f in api.files.query(
                limit = 100,project = my_project).all()]

        if f.name in my_files:
            print('Sucessfully copied one file!')
        else:
            print('Something went wrong...')

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/copy-a-file)